## Load dataset

In [ ]:
import os

from datasets import Audio, load_dataset

SIZE = None

ds = load_dataset(
    "OrcinusOrca/YouTube-Cantonese",
    split="train",
    streaming=False,
    num_proc=os.cpu_count() * 2,
)
ds = ds.shuffle(seed=42)
ds = ds.select(range(SIZE)) if SIZE else ds
ds = ds.cast_column("mp3", Audio(sampling_rate=16000))
ds

Resolving data files:   0%|          | 0/45 [00:00<?, ?it/s]

Resolving data files:   0%|          | 0/45 [00:00<?, ?it/s]

Loading dataset shards:   0%|          | 0/112 [00:00<?, ?it/s]

Dataset({
    features: ['mp3', 'json', '__key__', '__url__'],
    num_rows: 236576
})

In [ ]:
def process_columns(row):
    # Extract all text segments and join them into a single string
    row["caption"] = row["json"]["caption"]
    return row


ds = ds.map(
    process_columns,
    remove_columns=["json", "__key__", "__url__"],
    num_proc=os.cpu_count(),
)
ds

Dataset({
    features: ['mp3', 'caption'],
    num_rows: 236576
})

In [ ]:
import re


def is_spoken_cantonese(string: str) -> bool:
    """Filter strings that contain Cantonese-specific characters."""
    # Cantonese-specific characters
    PATTERNS = [
        "嘅",
        "啲",
        "㗎",
        "嗰",
        "喺",
        "咗",
        "搵",
        "睇",
        "咩",
        "嚟",
        "佢",
        "乜",
        "嘢",
        "畀",
        "唔",
        "喇",
        "哋",
        "咋",
        "嘛",
        "囉",
        "喎",
        "吓",
        "啱",
        "冇",
        "咁",
        "晒",
        "諗",
        "搞",
        "咪",
        "攰",
        "呃",
        "嗌",
        "拗",
        "嘈",
        "啖",
        "掟",
        "噉",
        "𥄫",
        "嚿",
        "叻",
        "拎",
        "咯",
        "乸",
        "噏",
        "屙",
        "嘥",
        "揸",
        "屌",
        "鳩",
        "閪",
        "撚",
        "柒",
        "𨳒",
        "𨳍",
        "𨳊",
        "𨶙",
        "同埋",
        "你係",
        "我係",
        "邊度",
        "邊個",
        "唔駛",
        "呢度",
        "點算",
        "返去",
        "返來",
        "翻去",
        "翻來",
        "加埋",
        "屋企",
        "仆街",
        "仲要",
    ]

    return any(re.search(pattern, string) for pattern in PATTERNS)


ds = ds.filter(
    lambda batch: [not is_spoken_cantonese(caption) for caption in batch["caption"]],
    batched=True,
    batch_size=os.cpu_count() * 1000,
)
ds

Dataset({
    features: ['mp3', 'caption'],
    num_rows: 213865
})

In [ ]:
def normalize_caption(string: str) -> str:
    """Remove all parentheses and their contents from a string and filter garbage characters."""
    # Remove parentheses and square brackets and the content inside
    string = re.sub(r"[\(\[].*?[\)\]]", "", string)

    # Filter invalid characters
    ENGLISH_PATTERN = r"([a-zA-Z])"
    CHINESE_PATTERN = r"([\u4e00-\u9fff])"
    DIGIT_PATTERN = r"(\d)"

    def is_valid_char(c):
        return re.match(ENGLISH_PATTERN, c) or re.match(CHINESE_PATTERN, c) or re.match(DIGIT_PATTERN, c) or c == " "

    string = "".join(c for c in string if is_valid_char(c))

    # Add space between English and Chinese characters
    string = re.sub(f"{ENGLISH_PATTERN}{CHINESE_PATTERN}", r"\1 \2", string)
    string = re.sub(f"{CHINESE_PATTERN}{ENGLISH_PATTERN}", r"\1 \2", string)

    # Remove multiple spaces
    string = re.sub(r" {2,}", " ", string)
    string = string.strip()
    return string


ds = ds.map(
    lambda batch: {"caption": [normalize_caption(caption) for caption in batch["caption"]]},
    batched=True,
    batch_size=os.cpu_count() * 1000,
    num_proc=os.cpu_count() * 2,
)
ds = ds.filter(
    lambda batch: [caption.strip() != "" for caption in batch["caption"]],
    batched=True,
    batch_size=os.cpu_count() * 1000,
)
ds

Dataset({
    features: ['mp3', 'caption'],
    num_rows: 213786
})

In [ ]:
from opencc import OpenCC


def s2hk(content: str) -> str:
    """Convert Simplified Chinese to Traditional Chinese."""
    return OpenCC("s2hk").convert(content)


ds = ds.map(
    lambda batch: {"caption": [s2hk(caption) for caption in batch["caption"]]},
    batched=True,
    batch_size=64,
    num_proc=os.cpu_count() * 2,
)
ds

Dataset({
    features: ['mp3', 'caption'],
    num_rows: 213786
})

In [ ]:
import random

from rich import print

print(ds.select(random.sample(range(len(ds)), 10))["caption"])

[
    '鮮味小菜的方法 嘩 米是脹鼓鼓的 越吃越甜 裡面的是沖繩苦瓜 嘩很漂亮呢 今集周圍黐餐 我們跟 May 去元朗 
即摘港產沖繩苦瓜和牛奶南瓜來煮餸 好好吃 我覺得自己在日本',
    '但過了三天 即2018年8月20日 陳慧文妹妹因未能聯絡到姐姐 以及 Nancy 一直也沒有媽媽消息 聯絡不到她媽媽 也開始擔心 
因為之前媽媽離家也會跟他們聯絡的 而且畢竟也是自己跟媽媽吵爭 才導致她離家出走 於是當天張祺忠一家就 前往警署報警 
事情去到這裡 是否覺得跟之前探員 X 調查的 周凱亮案件很相似 那宗案件 也是因為父母失蹤 之後兩名兒子報警',
    '首先是採用全新 Ada Lovelace 核心架構 升級至第四代的 Tensor Core 加入 Optical Flow Accelerator 
而光線追蹤核心則提升至第三代 打開顯示卡的外殼 便可見其 GPU 晶片上寫上 AD102300A1的型號 即為 RTX 4090的 GPU 晶片 
及其大容量記憶體 另外是支援 DLSS 30 以及新增 AV1編碼器',
    '接下來根據陳金兒的説法 就是法師帶上她 這個並不是訪問內容 是後來我觀看大佬 B 然後我現在綜合整個故事告訴你們 
後來大佬 B 即邵子風的網台 名為泰國油麻地就訪問陳金兒 根據陳金兒再補充的説法是',
    '法律諮詢探訪在囚人士等等 2019年反修例運動期間亦多次公開呼籲眾籌 他們的社交平台最後更新是今年6月 2019年底 
滙豐銀行以星火同盟的户口運作 與開户陳述時不同為理由 取消其户口 一個月後',
    '大家覺得哪個更好 用回舊的阿豬 升級版靚仔阿豬 還是你想要我 請你在 comment 留言 既然智能功能 
可以做完每一樣我要做的工作 那它會不會取代阿豬 沒有或者暫時沒有 以前我用10個小時做一條片 現在有 AI 的幫助 
可能6個小時就可以了 多出來的4個小時 我就用來做有用的事情 例如分析市場 所以當我們看回人類這麼多年歷史 
每一次有新的機械或者電腦',
    '如果沒有中風 它不影響你日常生活 是不需要處理 兩成人是會有這個閉不上的洞 如果不幸地 是六十歲以下 某時某月某日 
某個病人中風 又解釋不到 沒有糖尿病血壓高或膽固醇高的問題 又不吸煙 甚麼血管風險病變都沒有 便要找出原因 第一 
檢查他有沒有心律不正 心房顫動有最高風險中風',
    '它這個是屬於強刺激的 這個刺激很強 例如我們有時候發現有些面癱的病人 他的嘴會歪到一邊 我們也會在他的這個位置針灸
也會為他改善歪嘴的情況 督脈就是這幾個 大椎百會 對 那麼任脈呢 至於任脈一定要介紹我們稱為關元穴和氣海穴',
    '文心一言的技術還未很完善 同時也表示 AI 的英文能力 仍然有好大的進步空間 網民好奇地 文心一言 會怎樣 生成 
煲仔飯菠蘿包 和雞尾包等等的圖像 今個星期我們都有關於實試 ChatGPT4 的新功能資料 還有香港區課金玩 ChatGPT4 教學 另外 
AI 繪圖方面 我們都有 Midjourney V5 實例測試和使用方法教學',
    '東升西降我同不同意呢 我絕對同意 大家看到是什麼事情呢 就是在説中國的 GDP 就是在説中國的 GDP 從以前排在很後的 
現在去到全世界 現在去到全世界 是佔第二位 僅僅是跟著美國 東升西降是先行 但是又很不幸地你會看到 無論中國的股票市場 
或者是香港的股票市場'
]

## Load Models

### Processor (Feature Extractor & Tokenizer)

In [ ]:
from transformers import WhisperFeatureExtractor, WhisperProcessor, WhisperTokenizer

model_id = "openai/whisper-large-v3-turbo"
processor: WhisperProcessor = WhisperProcessor.from_pretrained(model_id, task="transcribe")
feature_extractor: WhisperFeatureExtractor = processor.feature_extractor
tokenizer: WhisperTokenizer = processor.tokenizer

## Prepare Dataset

### Tokenization

In [ ]:
def prepare_batch(batch: dict[str, list]) -> dict[str, list]:
    """Extract log-mels and tokenize captions for a batched slice of the dataset."""
    # Collect raw audio arrays and sample rate
    arrays = [audio["array"] for audio in batch["mp3"]]
    sr = batch["mp3"][0]["sampling_rate"]

    # 1) Feature extraction → returns a BatchFeature with .input_features: np.ndarray
    batch["input_features"] = processor.feature_extractor(
        arrays,
        sampling_rate=sr,
    ).input_features

    # 2) Tokenization of target captions → returns a BatchEncoding
    batch["labels"] = processor.tokenizer(
        batch["caption"],
    ).input_ids

    return batch


ds = ds.map(
    prepare_batch,
    batched=True,
    batch_size=os.cpu_count(),
    num_proc=os.cpu_count(),
    remove_columns=["mp3", "caption"],
)

Map (num_proc=72):   0%|          | 0/213786 [00:00<?, ? examples/s]

2025-05-08 03:13:20.630291: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-08 03:13:20.630441: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-08 03:13:20.630517: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-08 03:13:20.630577: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-05-08 03:13:20.631147: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for 

## Save

In [ ]:
ds_name = f"ds_{SIZE}" if SIZE else "ds"
ds.save_to_disk(f"dataset_local/{ds_name}", num_proc=os.cpu_count() * 2)

Saving the dataset (0/658 shards):   0%|          | 0/213786 [00:00<?, ? examples/s]